https://github.com/monologg/KoELECTRA?tab=readme-ov-file

# 0) 모듈 실행

##기본 모듈

In [1]:
# Matplotlib 와 pandas 그래프에서 한글 사용 설정
import sys

## 노트북이 코랩에서 실행중인지 체크
if 'google.colab' in sys.modules:
  !echo 'debconf debconf/frontend select Noninterative' | debconf-set-selections
  ## 나눔 폰트 설치
  !sudo apt-get -qq -y install fonts-nanum
  import matplotlib.font_manager as fm
  font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])
  for fpath in font_files:
    fm.fontManager.addfont(fpath)

  # 나눔바른고딕 폰트 설정
  import matplotlib.pyplot as plt
  plt.rc('font', family='NanumBarunGothic')
  # 마이너스 기호 표시 오류 수정
  import matplotlib
  matplotlib.rcParams['axes.unicode_minus'] = False


debconf: unable to initialize frontend: Noninterative
debconf: (Can't locate Debconf/FrontEnd/Noninterative.pm in @INC (you may need to install the Debconf::FrontEnd::Noninterative module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.34.0 /usr/local/share/perl/5.34.0 /usr/lib/x86_64-linux-gnu/perl5/5.34 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl-base /usr/lib/x86_64-linux-gnu/perl/5.34 /usr/share/perl/5.34 /usr/local/lib/site_perl) at (eval 17) line 2, <> line 1.)
debconf: falling back to frontend: Noninteractive
Selecting previously unselected package fonts-nanum.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [2]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 17.6 MB/s eta 0:00:00


In [3]:
!apt-get update -qq

In [4]:
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum-coding.
(Reading database ... 121941 files and directories currently installed.)
Preparing to unpack .../fonts-nanum-coding_2.5-3_all.deb ...
Unpacking fonts-nanum-coding (2.5-3) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-7_all.deb ...
Unpacking fonts-nanum-eco (1.000-7) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20200506-1_all.deb ...
Unpacking fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum-coding (2.5-3) ...
Setting up fonts-nanum-eco (1.000-7) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [5]:
import matplotlib.font_manager as fm
sys_font = fm.findSystemFonts()

# 한글 폰트 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
#fm._rebuild()

NanumBarunGothic


In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import random
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from tqdm import tqdm
import torch.nn.functional as F

In [8]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

## 모델링

In [9]:
# GPU 할당 확인
!nvidia-smi

Sun May 26 14:02:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
!pip install transformers pytorch_lightning tokenizers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 11.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [11]:
import tqdm
import torch
import tensorflow as tf

from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_scheduler
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [12]:
from transformers import AutoTokenizer
from transformers import ElectraForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from torch.nn import functional as F
from tqdm.notebook import tqdm

# 1) 데이터 가져오기

## 감성대화 말뭉치
https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86
- 일반인 1,500명 대상의 인터뷰 및 크라우드 소싱 수행
- 우울증 환자 대상 WOZ 대화 수집

=> 전처리 과정으로 "한사람만의 대화를 줄글로 변형 - 질문 제거"

### 1. 데이터 가져오기

In [13]:
train = pd.read_excel("/content/drive/MyDrive/Dataset/AIhub_감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.xlsx")
test = pd.read_excel("/content/drive/MyDrive/Dataset/AIhub_감성대화/Validation_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Validation.xlsx")

In [14]:
train_df = train.copy()
test_df = test.copy()

### 2. 전처리

In [15]:
# "사람문장3" 결측값 대체
train_df.fillna('',inplace=True)
test_df.fillna('',inplace=True)

# 문장 병합
train_df['사람문장'] = train_df['사람문장1']+ " " + train_df['사람문장2']+ " " +train_df['사람문장3']
test_df['사람문장'] = test_df['사람문장1']+ " " +test_df['사람문장2']+ " " +test_df['사람문장3']

# "감정 분류"과 "문장"만 추출
train_df = train_df[['사람문장', '감정_대분류']]
test_df = test_df[['사람문장', '감정_대분류']]

# 컬럼명 변경
train_df.rename(columns={'사람문장': 'text',
                   '감정_대분류': 'sentiment'}, inplace=True)
test_df.rename(columns={'사람문장': 'text',
                   '감정_대분류': 'sentiment'}, inplace=True)

# label encoding
le = LabelEncoder()
le=le.fit(train_df['sentiment'])
train_df['target']=le.transform(train_df['sentiment'])

le=le.fit(test_df['sentiment'])
test_df['target']=le.transform(test_df['sentiment'])

# text, target
train_conv = train_df[['text','target']]
test_conv = test_df[['text','target']]

<ipython-input-15-1d7e370bec27>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.rename(columns={'사람문장': 'text',
<ipython-input-15-1d7e370bec27>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.rename(columns={'사람문장': 'text',
<ipython-input-15-1d7e370bec27>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['target']=le.transform(train_df['sentiment'])


In [16]:
# 라벨 인코딩 결과 확인
sentiment = train_df[['sentiment', 'target']].drop_duplicates(subset=['sentiment']) # sentiment 컬럼에 대해 중복된 값을 갖고 있는 열 제거
df_unique = sentiment.sort_values(by='target', ascending=True)
df_unique = df_unique.reset_index(drop=True)
df_unique

,sentiment,target
0,기쁨,0
1,당황,1
2,분노,2
3,불안,3
4,상처,4
5,슬픔,5


# 2) 데이터 증강
데이터 불균형으로 인해 분류 전에 전처리하는 방식인 smote 택함

**SMOTE(= Synthetic Minority Over-sampling Technique)**

SMOTE는 합성을 기반으로 하는 방법으로, minority class의 샘플을 가져와 이들을 잇는 선에서 중간값을 택하여 만들어내는 방식

Train

In [17]:
# SMOTE : 오버샘플링
from imblearn.over_sampling import RandomOverSampler
text = train_conv.text.to_numpy().reshape(-1, 1)
target = train_conv.target.to_numpy().reshape(-1, 1)

oversample = RandomOverSampler()

X_oversample, y_oversample = oversample.fit_resample(text, target)
train_conv2 = pd.DataFrame({"text": X_oversample.reshape(-1), 'target': y_oversample.reshape(-1)})
train_conv2

,text,target
0,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...,2
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,2
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,2
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,2
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...,2
...,...,...
55915,낚시 동호회에서 만난 사람이 갑작스런 사고로 죽어서 친해질 기회를 잃었어. 요즘은 ...,5
55916,업무가 많아 너무 스트레스야. 나한테만 업무를 많이 시켜서 너무 스트레스를 받아.,5
55917,내 나이 오십 다 돼가는데 몸이 아파서 일을 못 해. 이게 잘못하면 장애 판정까지 ...,5
55918,미대 입시를 준비하고 있는 나보다 친구가 대충 그린 그림이 더 나아서 낙담했어. 내...,5


Test

In [18]:
# SMOTE : 오버샘플링
from imblearn.over_sampling import RandomOverSampler
text = test_conv.text.to_numpy().reshape(-1, 1)
target = test_conv.target.to_numpy().reshape(-1, 1)

oversample = RandomOverSampler()

X_oversample, y_oversample = oversample.fit_resample(text, target)
test_conv2 = pd.DataFrame({"text": X_oversample.reshape(-1), 'target': y_oversample.reshape(-1)})
test_conv2

,text,target
0,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,3
1,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워. 나에게 너무 크...,3
2,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워. 무섭게 생겼는데도 업무를 보려...,3
3,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다. 첫 직장...,3
4,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼. 내가 낯가림이 심해서 친해질 수...,3
...,...,...
7537,시대가 바뀌면서 내가 잘 알지 못하는 분야의 일까지 맡게 되었는데 적응이 안 돼서 ...,5
7538,아내가 돈 때문에 무릎 퇴행성관절염을 방치하다가 큰 수술을 받게 됐어. 돈이 뭔지 ...,5
7539,이번에 또 최종면접에서 떨어졌어. 이만큼 떨어졌는데 또 떨어지려나 생각했어. 너무 ...,5
7540,세상 사는 게 힘들어서 죽어 버리고 싶다는 생각이 들어. 맞아. 요즘 사는 게 너무...,5


# 3) 데이터 전처리
모델 적합을 위해 데이터가 적절히 전처리되어야 하며 이때 전처리는 토큰화, 정수 인코딩, 패딩을 의미함

In [19]:
import re
# 텍스트 클린징 함수 정의
def clean_text(text):
    # 특수문자 제거
    text = re.sub(r'[^가-힣A-Za-z0-9 ]', '', text)

    # 여러 개의 공백은 하나로 줄이기
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [20]:
train_df = train_conv2.copy()
test_df = test_conv2.copy()

train_df['text'] = train_df['text'].apply(clean_text)
test_df['text'] = test_df['text'].apply(clean_text)

In [21]:
train_df

,text,target
0,일은 왜 해도 해도 끝이 없을까 화가 난다 그냥 내가 해결하는 게 나아 남들한테 부...,2
1,이번 달에 또 급여가 깎였어 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나 최...,2
2,회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...,2
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은 데 정말 분하고 섭...,2
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나 상사인 나에게 먼...,2
...,...,...
55915,낚시 동호회에서 만난 사람이 갑작스런 사고로 죽어서 친해질 기회를 잃었어 요즘은 부...,5
55916,업무가 많아 너무 스트레스야 나한테만 업무를 많이 시켜서 너무 스트레스를 받아,5
55917,내 나이 오십 다 돼가는데 몸이 아파서 일을 못 해 이게 잘못하면 장애 판정까지 받...,5
55918,미대 입시를 준비하고 있는 나보다 친구가 대충 그린 그림이 더 나아서 낙담했어 내 ...,5


토큰화
- KoElectra의 토크나이저를 쓰기 위해선 from_pretrained 함수의 인자로 monologg/koelectra-small-v3-discriminator 를 넣음

- ##이 같은 것들은 BERT나 ELECTRA 같은 토크나이저의 특징 중 하나
- 단어를 더 잘 이해할 수 있도록 도움

In [22]:
# 데이터셋 클래스: SentimentReviewDataset 클래스를 사용하여 입력 데이터를 준비
from transformers import ElectraTokenizer
from torch.utils.data import DataLoader, Dataset

class SentimentReviewDataset(Dataset):

  def __init__(self, dataset):
    self.dataset = dataset
    self.tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, ].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text,
        return_tensors='pt', #pytorch 사용
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
        )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [23]:
train_dataset = SentimentReviewDataset(train_df)
test_dataset = SentimentReviewDataset(test_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

In [24]:
train_dataset[2]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(tensor([    2,  6387,  4073, 10390,  4007, 10199, 18781, 14643,  4070,  2041,
          4525,  4228,  6269,  3093,  4110,  7465,  8070,  6556,  6243, 20653,
          4149,  8692, 29895,  3258,  3081,  2639,  4034,  6226,  4007,  4294,
         14419,  3258,  8730,  4034,  2048,  4275,  4071,  6681,  4112,  6713,
          4110, 29252, 10766,  4110, 10855,  4129,  8028,  4034,  2054,  2376,
          8692,  4736,  4292,  2048,  2024,  4153,     3,     0,     0,     0,
             0,     0,     0,     0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]),
 2)

# 4) 모델 생성

In [25]:
# GPU 사용
device = torch.device("cuda")

In [26]:
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
#model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
'''from transformers import ElectraModel, ElectraTokenizer

model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")'''

'from transformers import ElectraModel, ElectraTokenizer\n\nmodel = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")\ntokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")'

In [28]:
# 한번 실행해보기
'''text, attention_mask, y = train_dataset[0]
model(text.unsqueeze(0), attention_mask = attention_mask.unsqueeze(0))'''

'text, attention_mask, y = train_dataset[0]\nmodel(text.unsqueeze(0), attention_mask = attention_mask.unsqueeze(0))'

In [29]:
# 분류 헤드 수정 - KoElectra 모델의 출력층을 6개의 클래스로 분류할 수 있도록 수정
import torch.nn as nn

model.classifier.out_proj = nn.Linear(in_features=768, out_features=6)

In [30]:
# 모델 레이어 보기
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [31]:
# 레이블 값 확인 : 레이블 값이 올바른 범위 내에 있는지 확인
'''print(y_batch.unique())
# 모델 출력 형태 확인
print(y_pred.shape)
print(y_batch.shape)'''

'print(y_batch.unique())\n# 모델 출력 형태 확인\nprint(y_pred.shape)\nprint(y_batch.shape)'

# 5) 모델 학습

In [32]:
epochs = 5
batch_size = 32

optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
losses = []
accuracies = []

# 모델을 올바른 디바이스로 이동
model.to(device)

for i in range(epochs):
    total_loss = 0.0
    correct = 0
    total = 0
    batches = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        # 배치 데이터를 GPU로 이동
        input_ids_batch = input_ids_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()

        # 모델 예측
        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        y_pred = outputs.logits

        # 손실 계산
        loss = F.cross_entropy(y_pred, y_batch) # torch.nn.functiona
        # 역전파
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 계산
        total_loss += loss.item()
        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

        batches += 1
        if batches % 100 == 0:
            print("Batch Loss:", total_loss, "Accuracy:", correct / total)

    losses.append(total_loss)
    accuracies.append(correct / total)
    print(f"Epoch {i+1}, Train Loss: {total_loss}, Accuracy: {correct / total}")

  0%|          | 0/1748 [00:00<?, ?it/s]

Batch Loss: 178.86350858211517 Accuracy: 0.17125
Batch Loss: 339.13830399513245 Accuracy: 0.25265625
Batch Loss: 480.62654089927673 Accuracy: 0.31302083333333336
Batch Loss: 608.9382005929947 Accuracy: 0.361328125
Batch Loss: 727.2829815149307 Accuracy: 0.401125
Batch Loss: 841.1503506302834 Accuracy: 0.4315104166666667
Batch Loss: 948.5787113904953 Accuracy: 0.4576339285714286
Batch Loss: 1052.266095817089 Accuracy: 0.479765625
Batch Loss: 1150.7904310822487 Accuracy: 0.4990277777777778
Batch Loss: 1254.768340408802 Accuracy: 0.512125
Batch Loss: 1354.0770522356033 Accuracy: 0.5253409090909091
Batch Loss: 1453.1200355887413 Accuracy: 0.53515625
Batch Loss: 1550.5605010390282 Accuracy: 0.5439663461538462
Batch Loss: 1645.4623882770538 Accuracy: 0.5522544642857142
Batch Loss: 1741.1068451404572 Accuracy: 0.5593125
Batch Loss: 1834.6824542880058 Accuracy: 0.56630859375
Batch Loss: 1926.7593341469765 Accuracy: 0.5726102941176471
Epoch 1, Train Loss: 1970.0755832791328, Accuracy: 0.5755901

  0%|          | 0/1748 [00:00<?, ?it/s]

Batch Loss: 90.29197871685028 Accuracy: 0.6809375
Batch Loss: 175.65619438886642 Accuracy: 0.69078125
Batch Loss: 261.4075449705124 Accuracy: 0.6951041666666666
Batch Loss: 350.40172332525253 Accuracy: 0.692890625
Batch Loss: 436.5233246386051 Accuracy: 0.6938125
Batch Loss: 521.0633941590786 Accuracy: 0.6952083333333333
Batch Loss: 607.737181276083 Accuracy: 0.6952232142857143
Batch Loss: 686.1624892652035 Accuracy: 0.698671875
Batch Loss: 769.7343367934227 Accuracy: 0.6984027777777778
Batch Loss: 851.3806565403938 Accuracy: 0.6999375
Batch Loss: 936.9350909292698 Accuracy: 0.6998579545454545
Batch Loss: 1020.9832489490509 Accuracy: 0.6997916666666667
Batch Loss: 1103.7744731903076 Accuracy: 0.6996634615384615
Batch Loss: 1188.4544846117496 Accuracy: 0.6998660714285714
Batch Loss: 1272.6185320317745 Accuracy: 0.6999166666666666
Batch Loss: 1355.6946525275707 Accuracy: 0.700234375
Batch Loss: 1439.6268248856068 Accuracy: 0.7001286764705882
Epoch 2, Train Loss: 1479.0548672676086, Accur

  0%|          | 0/1748 [00:00<?, ?it/s]

Batch Loss: 77.09773875772953 Accuracy: 0.7346875
Batch Loss: 152.87550021708012 Accuracy: 0.7296875
Batch Loss: 228.2069738060236 Accuracy: 0.7348958333333333
Batch Loss: 304.3782456666231 Accuracy: 0.733203125
Batch Loss: 379.79693888127804 Accuracy: 0.733375
Batch Loss: 455.62317346036434 Accuracy: 0.7334895833333334
Batch Loss: 530.230910167098 Accuracy: 0.7340625
Batch Loss: 607.7031712383032 Accuracy: 0.7334765625
Batch Loss: 686.0320280939341 Accuracy: 0.7323611111111111
Batch Loss: 759.5720108896494 Accuracy: 0.7335
Batch Loss: 833.9211164563894 Accuracy: 0.7344602272727273
Batch Loss: 905.7003039568663 Accuracy: 0.73515625
Batch Loss: 980.6676793843508 Accuracy: 0.7355769230769231
Batch Loss: 1059.5018235892057 Accuracy: 0.7345758928571429
Batch Loss: 1138.1059476286173 Accuracy: 0.7341458333333334
Batch Loss: 1216.4259711056948 Accuracy: 0.73361328125
Batch Loss: 1292.2041850239038 Accuracy: 0.7334191176470588
Epoch 3, Train Loss: 1326.2539757043123, Accuracy: 0.7338340486409

  0%|          | 0/1748 [00:00<?, ?it/s]

Batch Loss: 70.60044130682945 Accuracy: 0.754375
Batch Loss: 138.22503888607025 Accuracy: 0.75921875
Batch Loss: 207.8856175839901 Accuracy: 0.7576041666666666
Batch Loss: 274.03229609131813 Accuracy: 0.76109375
Batch Loss: 343.7886322438717 Accuracy: 0.7596875
Batch Loss: 414.48896837234497 Accuracy: 0.7580729166666667
Batch Loss: 484.0841394215822 Accuracy: 0.7585267857142857
Batch Loss: 550.9896504729986 Accuracy: 0.759765625
Batch Loss: 619.9205550700426 Accuracy: 0.7595833333333334
Batch Loss: 689.9302027374506 Accuracy: 0.75903125
Batch Loss: 757.834215387702 Accuracy: 0.7592329545454546
Batch Loss: 823.6032979041338 Accuracy: 0.7602083333333334
Batch Loss: 890.2285696417093 Accuracy: 0.7608173076923077
Batch Loss: 958.2611132115126 Accuracy: 0.7601785714285715
Batch Loss: 1027.268230304122 Accuracy: 0.7601666666666667
Batch Loss: 1095.9221639484167 Accuracy: 0.76060546875
Batch Loss: 1163.0719646066427 Accuracy: 0.7607352941176471
Epoch 4, Train Loss: 1194.9146396368742, Accurac

  0%|          | 0/1748 [00:00<?, ?it/s]

Batch Loss: 60.087137535214424 Accuracy: 0.7953125
Batch Loss: 120.1252049356699 Accuracy: 0.791875
Batch Loss: 181.16259461641312 Accuracy: 0.7898958333333334
Batch Loss: 244.08783140778542 Accuracy: 0.78984375
Batch Loss: 302.11728700995445 Accuracy: 0.7926875
Batch Loss: 362.12119603157043 Accuracy: 0.79125
Batch Loss: 421.2761610150337 Accuracy: 0.7916964285714285
Batch Loss: 481.5776849091053 Accuracy: 0.79125
Batch Loss: 541.5902387797832 Accuracy: 0.7922916666666666
Batch Loss: 606.1928136646748 Accuracy: 0.7910625
Batch Loss: 669.0516258180141 Accuracy: 0.7899431818181818
Batch Loss: 730.1354731321335 Accuracy: 0.789453125
Batch Loss: 791.6699852049351 Accuracy: 0.7896875
Batch Loss: 850.6094172596931 Accuracy: 0.7902901785714286
Batch Loss: 911.0395115315914 Accuracy: 0.7906041666666667
Batch Loss: 971.4701105207205 Accuracy: 0.790390625
Batch Loss: 1031.6378740817308 Accuracy: 0.7903308823529411
Epoch 5, Train Loss: 1062.2551193088293, Accuracy: 0.7902539341917024


In [35]:
'''losses = []
accuracies = []

# 모델을 올바른 디바이스로 이동
model.to(device)

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
     # 배치 데이터를 GPU로 이동
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    # 손실 및 정확도 계산
    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)

  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)'''

'losses = []\naccuracies = []\n\n# 모델을 올바른 디바이스로 이동\nmodel.to(device)\n\nfor i in range(epochs):\n  total_loss = 0.0\n  correct = 0\n  total = 0\n  batches = 0\n\n  model.train()\n\n  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):\n     # 배치 데이터를 GPU로 이동\n    optimizer.zero_grad()\n    y_batch = y_batch.to(device)\n    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]\n    loss = F.cross_entropy(y_pred, y_batch)\n    loss.backward()\n    optimizer.step()\n\n    # 손실 및 정확도 계산\n    total_loss += loss.item()\n\n    _, predicted = torch.max(y_pred, 1)\n    correct += (predicted == y_batch).sum()\n    total += len(y_batch)\n\n    batches += 1\n    if batches % 100 == 0:\n      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)\n\n  losses.append(total_loss)\n  accuracies.append(correct.float() / total)\n  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)'

In [36]:
losses, accuracies

([1970.0755832791328,
  1479.0548672676086,
  1326.2539757043123,
  1194.9146396368742,
  1062.2551193088293],
 [0.5755901287553649,
  0.7001788268955651,
  0.7338340486409156,
  0.7606759656652361,
  0.7902539341917024])

In [37]:
# 모델 저장하기
model_path = "/content/drive/MyDrive/model_KoElectra_0523.pt"
torch.save(model.state_dict(), model_path)

# 6) 모델 평가

In [38]:
from sklearn.metrics import classification_report

In [39]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/236 [00:00<?, ?it/s]

Accuracy: tensor(0.7548, device='cuda:0')


# 7) 모델 추가 학습

- 저장된 모델을 불러온 후, 추가학습을 통해 모델 성능을 더 향상시킴
- 모델을 점진적으로 개선하고 데이터와 학습 조건을 조정하며 성능 최적화

In [40]:
model.classifier.out_proj = nn.Linear(in_features=768, out_features=6)
model.load_state_dict(torch.load(model_path))
model.to(device)

# 옵티마이저 설정
epochs = 5
batch_size = 16
optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# 추가 학습 루프
for i in range(epochs):  # 추가 학습할 에포크 수
    total_loss = 0.0
    correct = 0
    total = 0
    batches = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        # 배치 데이터를 GPU로 이동
        input_ids_batch = input_ids_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()

        # 모델 예측
        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        y_pred = outputs.logits

        # 손실 계산
        loss = F.cross_entropy(y_pred, y_batch)

        # 역전파
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 계산
        total_loss += loss.item()
        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

        batches += 1
        if batches % 100 == 0:
            print("Batch Loss:", total_loss, "Accuracy:", correct / total)

    print(f"Epoch {i+1}, Train Loss: {total_loss}, Accuracy: {correct / total}")

  0%|          | 0/3495 [00:00<?, ?it/s]

Batch Loss: 53.02234340459108 Accuracy: 0.8225
Batch Loss: 102.21823079138994 Accuracy: 0.82625
Batch Loss: 149.3516907542944 Accuracy: 0.8295833333333333
Batch Loss: 202.9701580181718 Accuracy: 0.82484375
Batch Loss: 256.6107064411044 Accuracy: 0.8215
Batch Loss: 313.42648985236883 Accuracy: 0.8198958333333334
Batch Loss: 368.19619649648666 Accuracy: 0.8197321428571429
Batch Loss: 425.6571678072214 Accuracy: 0.817578125
Batch Loss: 480.820869974792 Accuracy: 0.8170138888888889
Batch Loss: 533.447565253824 Accuracy: 0.81725
Batch Loss: 588.6766639016569 Accuracy: 0.8165340909090909
Batch Loss: 646.8112869523466 Accuracy: 0.8153645833333333
Batch Loss: 700.0406297855079 Accuracy: 0.8152403846153846
Batch Loss: 755.403424475342 Accuracy: 0.8158482142857143
Batch Loss: 814.504685241729 Accuracy: 0.814875
Batch Loss: 865.9419856183231 Accuracy: 0.8152734375
Batch Loss: 921.4387926794589 Accuracy: 0.8148161764705882
Batch Loss: 976.0320680178702 Accuracy: 0.8145138888888889
Batch Loss: 1030

  0%|          | 0/3495 [00:00<?, ?it/s]

Batch Loss: 45.87083873897791 Accuracy: 0.84625
Batch Loss: 92.7691577449441 Accuracy: 0.845625
Batch Loss: 140.32921722903848 Accuracy: 0.84375
Batch Loss: 187.73667603358626 Accuracy: 0.8415625
Batch Loss: 234.48055581375957 Accuracy: 0.8415
Batch Loss: 278.78411657735705 Accuracy: 0.8442708333333333
Batch Loss: 326.57971144095063 Accuracy: 0.8423214285714286
Batch Loss: 373.69668566808105 Accuracy: 0.84234375
Batch Loss: 420.49624610319734 Accuracy: 0.8421527777777778
Batch Loss: 465.9976110868156 Accuracy: 0.8430625
Batch Loss: 511.0567193366587 Accuracy: 0.8426136363636364
Batch Loss: 556.7145493887365 Accuracy: 0.8430208333333333
Batch Loss: 606.8406026475132 Accuracy: 0.8415384615384616
Batch Loss: 656.0946852900088 Accuracy: 0.8410267857142857
Batch Loss: 704.3979121856391 Accuracy: 0.8402916666666667
Batch Loss: 750.3765460886061 Accuracy: 0.840546875
Batch Loss: 792.208024341613 Accuracy: 0.8415073529411765
Batch Loss: 843.1588814072311 Accuracy: 0.8403472222222222
Batch Loss

  0%|          | 0/3495 [00:00<?, ?it/s]

Batch Loss: 41.79220174998045 Accuracy: 0.868125
Batch Loss: 77.06648025661707 Accuracy: 0.87625
Batch Loss: 115.45086794346571 Accuracy: 0.8739583333333333
Batch Loss: 154.51740572601557 Accuracy: 0.87453125
Batch Loss: 192.45207890868187 Accuracy: 0.87375
Batch Loss: 227.82204937934875 Accuracy: 0.8754166666666666
Batch Loss: 262.7614425010979 Accuracy: 0.8759821428571428
Batch Loss: 301.26816527172923 Accuracy: 0.87546875
Batch Loss: 345.7281719855964 Accuracy: 0.8731944444444445
Batch Loss: 387.43140273168683 Accuracy: 0.8718125
Batch Loss: 423.37923077866435 Accuracy: 0.8727272727272727
Batch Loss: 466.1099552549422 Accuracy: 0.8714583333333333
Batch Loss: 501.33098904043436 Accuracy: 0.871875
Batch Loss: 542.7657977715135 Accuracy: 0.87125
Batch Loss: 584.8377450332046 Accuracy: 0.870625
Batch Loss: 622.4407919980586 Accuracy: 0.871328125
Batch Loss: 662.7511806599796 Accuracy: 0.8709558823529412
Batch Loss: 702.1771974302828 Accuracy: 0.8707986111111111
Batch Loss: 741.785852793

  0%|          | 0/3495 [00:00<?, ?it/s]

Batch Loss: 31.46401782333851 Accuracy: 0.898125
Batch Loss: 58.60353620350361 Accuracy: 0.906875
Batch Loss: 88.9986111074686 Accuracy: 0.9064583333333334
Batch Loss: 118.15564068220556 Accuracy: 0.90578125
Batch Loss: 148.98131894133985 Accuracy: 0.905625
Batch Loss: 180.7287057992071 Accuracy: 0.904375
Batch Loss: 209.32810504175723 Accuracy: 0.9048214285714286
Batch Loss: 239.77280643768609 Accuracy: 0.903984375
Batch Loss: 273.40921001695096 Accuracy: 0.9021527777777778
Batch Loss: 302.8243873361498 Accuracy: 0.9026875
Batch Loss: 334.01570900529623 Accuracy: 0.9020454545454546
Batch Loss: 366.4953058473766 Accuracy: 0.9011458333333333
Batch Loss: 398.2878234088421 Accuracy: 0.9010096153846154
Batch Loss: 432.1655183546245 Accuracy: 0.8995089285714286
Batch Loss: 465.1975490693003 Accuracy: 0.89875
Batch Loss: 496.38958746753633 Accuracy: 0.8987890625
Batch Loss: 528.9568190891296 Accuracy: 0.8988970588235294
Batch Loss: 560.6530963499099 Accuracy: 0.8990277777777778
Batch Loss: 5

  0%|          | 0/3495 [00:00<?, ?it/s]

Batch Loss: 24.570235688239336 Accuracy: 0.918125
Batch Loss: 51.67992008663714 Accuracy: 0.9175
Batch Loss: 74.50077861174941 Accuracy: 0.9197916666666667
Batch Loss: 98.3975121024996 Accuracy: 0.920625
Batch Loss: 123.07470524124801 Accuracy: 0.92075
Batch Loss: 147.99351842142642 Accuracy: 0.92125
Batch Loss: 174.8100130185485 Accuracy: 0.9194642857142857
Batch Loss: 201.7185378037393 Accuracy: 0.9184375
Batch Loss: 227.88209012150764 Accuracy: 0.9179861111111111
Batch Loss: 257.9812679570168 Accuracy: 0.9165625
Batch Loss: 285.85864215530455 Accuracy: 0.9163068181818181
Batch Loss: 311.35131721384823 Accuracy: 0.9161979166666666
Batch Loss: 336.7653621342033 Accuracy: 0.9159615384615385
Batch Loss: 361.50480886828154 Accuracy: 0.91625
Batch Loss: 385.0941292559728 Accuracy: 0.9162916666666666
Batch Loss: 410.0492286765948 Accuracy: 0.91625
Batch Loss: 437.7112692184746 Accuracy: 0.9161029411764706
Batch Loss: 462.8163592033088 Accuracy: 0.9161805555555556
Batch Loss: 487.1776677276

In [41]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/472 [00:00<?, ?it/s]

Accuracy: tensor(0.7806, device='cuda:0')


In [44]:
# 모델 저장하기
model_path = "/content/drive/MyDrive/model_KoElectra_0527.pt"
torch.save(model.state_dict(), model_path)

In [45]:
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L